# Conversation starter


Adapted from: [LangChain docs](https://python.langchain.com/docs/get_started/quickstart)


If you are running this outside of [Dartmouth's JupyterHub](https://jhub.dartmouth.edu), make sure you follow the instructions in the [README](README.md#getting-started) to get started.


Prompting an LLM using LangChain


There are two types of language models, which in LangChain are called:

- LLMs: this is a language model which takes a string as input and returns a string
- ChatModels: this is a language model which takes a list of messages as input and returns a message


## Loading our API key


In [ ]:
from dotenv import load_dotenv
import os

load_dotenv("../secrets.env")

os.getenv("OPENAI_API_KEY") is not None

## Basic prompting using strings


In [ ]:
from langchain.llms import OpenAI

In [ ]:
llm = OpenAI(model="gpt-3.5-turbo-instruct")

In [ ]:
print(llm.predict("What is a Large Language Model?"))

In [ ]:
from langchain.chat_models import ChatOpenAI

In [ ]:
chat_model = ChatOpenAI(model="gpt-3.5-turbo")

In [ ]:
print(
    chat_model.predict(
        "What is the difference between a Large Language Model and a Chat Model?"
    )
)

## Prompting with `Message`s


In [ ]:
from langchain.schema import HumanMessage

In [ ]:
text = "What would be a good name for my dog?"
messages = [HumanMessage(content=text)]

In [ ]:
print(llm.predict_messages(messages).content)

In [ ]:
print(chat_model.predict_messages(messages).content)

## Prompt templates


In [ ]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("What is a good name for my {pet}?")

prompt.format(pet="lizard")

In [ ]:
pets = ["dog", "cat", "lizard", "goldfish"]
for pet in pets:
    print("*" * 10 + pet + "*" * 10)
    print(llm.predict(prompt.format(pet=pet)))

In [ ]:
for pet in pets:
    print("*" * 10 + pet + "*" * 10)
    print(chat_model.predict(prompt.format(pet=pet)))

In [ ]:
from langchain.prompts.chat import ChatPromptTemplate

system_template = "Respond to every prompt as if you were a {pet}."
human_template = "What is a good name for my {pet}?"

chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_template),
        ("human", human_template),
    ]
)

chat_prompt.format_messages(pet="dog")

In [ ]:
for pet in pets:
    print("*" * 10 + pet + "*" * 10)
    print(llm.predict_messages(chat_prompt.format_messages(pet=pet)).content)

In [ ]:
for pet in pets:
    print("*" * 10 + f" {pet} " + "*" * 10)
    print(chat_model.predict_messages(chat_prompt.format_messages(pet=pet)).content)

## Getting structured output for further processing


In [ ]:
from langchain.schema import BaseOutputParser


class CommaSeparatedListOutputParser(BaseOutputParser):
    """Parse the output of an LLM call to a comma-separated list."""

    def parse(self, text: str):
        """Parse the output of an LLM call."""
        return text.strip().split(", ")


parser = CommaSeparatedListOutputParser()
parser.parse("One, two")

In [ ]:
system_template = """You are a helpful assistant who generates comma separated lists. You ONLY return the comma separated lists, nothing else."""
human_template = "List 5 good names for my {pet}?"

chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_template),
        ("human", human_template),
    ]
)
for pet in pets:
    print(
        parser.parse(
            chat_model.predict_messages(chat_prompt.format_messages(pet=pet)).content
        )
    )

## Putting it all together

Creating a chain using the [LangChain Expression Language](https://python.langchain.com/docs/expression_language).


In [ ]:
chain = chat_prompt | chat_model | parser

In [ ]:
for pet in pets:
    print(chain.invoke({"pet": pet}))